In [18]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing, LagrangesMethod
from sympy.printing.octave import octave_code

class RobotMathEngineFast:
    def __init__(self, joint_config, link_vectors_mask):
        self.joint_config = joint_config
        # Converter para Matrix para evitar problemas de float
        self.link_vectors_mask = [sp.Matrix(v) for v in link_vectors_mask]

        self.t = sp.symbols('t')
        self.g = sp.symbols('g')

        self.q = []
        self.dq = []
        self.params_list = []

        self.frames = []
        self.rotation_matrices = []
        self.com_positions_global = []
        self.angular_velocities = []
        self.masses = []

        self.M = None
        self.G_vec = None
        self.C_total = None # Para 6-DOF, não vamos separar C e V (muito custoso)
        self.Jacobian = None

    def _rot_matrix_local(self, axis, angle):
        c, s = sp.cos(angle), sp.sin(angle)
        if axis == 'x': return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
        if axis == 'y': return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])
        if axis == 'z': return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
        return sp.eye(3)

    def _get_axis_vector(self, axis):
        if axis == 'x': return sp.Matrix([1, 0, 0])
        if axis == 'y': return sp.Matrix([0, 1, 0])
        if axis == 'z': return sp.Matrix([0, 0, 1])
        return sp.Matrix([0,0,0])

    def step_1_kinematics(self):
        print("1. Calculando Cinemática (Fast Mode)...")
        T_acc = sp.eye(4)
        R_acc = sp.eye(3)
        omega_acc = sp.Matrix([0, 0, 0])

        for i, (j_type, link_vec) in enumerate(zip(self.joint_config, self.link_vectors_mask)):
            q = dynamicsymbols(f'q{i+1}')
            dq = dynamicsymbols(f'q{i+1}', 1)
            self.q.append(q)
            self.dq.append(dq)

            m = sp.symbols(f'm{i+1}')
            L = sp.symbols(f'L{i+1}')
            cx, cy, cz = sp.symbols(f'cx{i+1} cy{i+1} cz{i+1}')
            self.masses.append(m)
            self.params_list.extend([m, L, cx, cy, cz])

            type_char, axis_char = j_type[0], j_type[1].lower()
            axis_vec_local = self._get_axis_vector(axis_char)

            # --- ROTAÇÃO ---
            if type_char == 'R':
                # SEM SIMPLIFY AQUI
                axis_vec_global = R_acc * axis_vec_local
                omega_new = omega_acc + axis_vec_global * dq
                R_j = self._rot_matrix_local(axis_char, q)
                P_j = sp.Matrix([0,0,0])
            elif type_char == 'D':
                omega_new = omega_acc
                R_j = sp.eye(3)
                P_j = sp.Matrix([0,0,0])
                if axis_char == 'x': P_j[0] = q
                if axis_char == 'y': P_j[1] = q
                if axis_char == 'z': P_j[2] = q

            # Atualiza Globais (Sem simplificar)
            R_acc = R_acc * R_j
            self.rotation_matrices.append(R_acc)
            self.angular_velocities.append(omega_new)
            omega_acc = omega_new

            # --- GEOMETRIA ---
            T_joint = sp.eye(4)
            T_joint[0:3, 0:3] = R_j
            T_joint[0:3, 3] = P_j
            T_at_joint_start = T_acc * T_joint

            v_com_local = sp.Matrix([cx, cy, cz, 1])
            p_com_global = T_at_joint_start * v_com_local
            self.com_positions_global.append(p_com_global[0:3, :])

            P_link_vec = link_vec * L
            T_link = sp.eye(4)
            T_link[0:3, 3] = P_link_vec

            T_acc = T_at_joint_start * T_link
            self.frames.append(T_acc)

    def step_2_jacobian(self):
        print("2. Calculando Jacobiano...")
        pos_ee = self.frames[-1][0:3, 3]
        self.Jacobian = pos_ee.jacobian(self.q)

    def step_3_dynamics(self):
        print("3. Calculando Dinâmica (Sem simplify)...")
        T_tot, V_tot = 0, 0

        iterator = zip(self.com_positions_global, self.masses, self.rotation_matrices, self.angular_velocities)

        for i, (pos_cm, m, R_global, w_global) in enumerate(iterator):
            # Translacional
            vel_linear = pos_cm.diff(self.t)
            # DICA DE VELOCIDADE: Usar dot() direto sem criar matrizes intermediarias desnecessarias
            T_trans = sp.Rational(1,2) * m * (vel_linear.T * vel_linear)[0]

            # Rotacional
            Ixx, Iyy, Izz = sp.symbols(f'Ixx{i+1} Iyy{i+1} Izz{i+1}')
            self.params_list.extend([Ixx, Iyy, Izz])
            I_local = sp.Matrix([[Ixx, 0, 0], [0, Iyy, 0], [0, 0, Izz]])

            # Matriz global crua (sem simplify)
            I_global = R_global * I_local * R_global.T
            T_rot = sp.Rational(1,2) * (w_global.T * I_global * w_global)[0]

            T_tot += (T_trans + T_rot)

            # Potencial
            V_tot += m * self.g * pos_cm[2]

        # AQUI É O SEGREDO: Não simplificamos o Lagrangeano.
        # Passamos a expressão "suja" direto para o derivador.
        self.lagrangian = T_tot - V_tot

        print("   Derivando Lagrangeano (Isso ainda demora um pouco)...")
        LM = LagrangesMethod(self.lagrangian, self.q)
        LM.form_lagranges_equations()

        print("   Extraindo Matrizes...")
        self.M = LM.mass_matrix

        # Para 6-DOF, Coriolis separado é MUITO lento. Vamos pegar o vetor total.
        # F = M*ddq + C_total => C_total = -F (assumindo sem gravidade no forcing)
        # O forcing do Sympy inclui gravidade se não estiver no lado direito.
        # Vamos usar o forcing direto e subtrair a gravidade depois.

        V_matrix = sp.Matrix([V_tot])
        self.G_vec = V_matrix.jacobian(self.q).T

        # Termo de forças cinéticas (Coriolis + Centrípeta)
        self.C_total = -LM.forcing - self.G_vec

    def step_4_export_cse_simulink(self):
        print("\n" + "="*40)
        print(" GERANDO CÓDIGO OTIMIZADO (CSE) ")
        print("="*40)
        print("Isso cria variáveis auxiliares (x0, x1...) para reduzir o cálculo.")

        # Limpeza de q(t) -> q
        mapa_subs = {}
        for i in range(len(self.q)):
            sym_q = sp.Symbol(f'q{i+1}')
            sym_dq = sp.Symbol(f'dq{i+1}')
            mapa_subs[self.q[i]] = sym_q
            mapa_subs[self.dq[i]] = sym_dq
            mapa_subs[self.q[i].diff(self.t)] = sym_dq

        # Agrupar expressões para otimização
        expressoes = [
            self.M.subs(mapa_subs),
            self.G_vec.subs(mapa_subs),
            self.C_total.subs(mapa_subs),
            self.Jacobian.subs(mapa_subs)
        ]

        # Common Subexpression Elimination
        # Retorna: [(x0, cos(q1)), (x1, sin(q1))...], [expr_simplificadas]
        replacements, reduced_exprs = sp.cse(expressoes)

        # --- IMPRESSÃO ---

        # 1. Variáveis Auxiliares
        print("% --- Variáveis Auxiliares (Cálculo Otimizado) ---")
        for sym, expr in replacements:
            print(f"{sym} = {octave_code(expr)};")

        # 2. Matrizes Finais (Usando as variaveis auxiliares)
        M_opt, G_opt, C_opt, J_opt = reduced_exprs

        print("\n% --- Matrizes Finais ---")
        print("M = " + octave_code(M_opt) + ";")
        print("G = " + octave_code(G_opt) + ";")
        # Coriolis Total (Centrifuga + Coriolis juntas, separadas é inviável em 6DOF simbólico rápido)
        print("V_total = " + octave_code(C_opt) + ";")
        print("J = " + octave_code(J_opt) + ";")

# --- TESTE RÁPIDO NO ANTROPOMÓRFICO ---
antropo_joints = ['Rz', 'Ry', 'Ry', 'Rz', 'Ry', 'Rz']
antropo_links = [[0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1]]

bot_fast = RobotMathEngineFast(antropo_joints, antropo_links)
bot_fast.step_1_kinematics()
bot_fast.step_2_jacobian()
bot_fast.step_3_dynamics()
bot_fast.step_4_export_cse_simulink()

1. Calculando Cinemática (Fast Mode)...
2. Calculando Jacobiano...
3. Calculando Dinâmica (Sem simplify)...
   Derivando Lagrangeano (Isso ainda demora um pouco)...
   Extraindo Matrizes...

 GERANDO CÓDIGO OTIMIZADO (CSE) 
Isso cria variáveis auxiliares (x0, x1...) para reduzir o cálculo.
% --- Variáveis Auxiliares (Cálculo Otimizado) ---
x0 = sin(q2);
x1 = x0.^2;
x2 = Ixx2.*x1;
x3 = cos(q3);
x4 = x0.*x3;
x5 = sin(q3);
x6 = cos(q2);
x7 = x5.*x6;
x8 = -x4 - x7;
x9 = x8.^2;
x10 = x6.^2;
x11 = Izz2.*x10;
x12 = x0.*x5;
x13 = x12 - x3.*x6;
x14 = -x13;
x15 = x14.^2;
x16 = cos(q1);
x17 = cx1.*x16;
x18 = sin(q1);
x19 = cy1.*x18;
x20 = 2*x17;
x21 = -2*x19 + x20;
x22 = cx1.*x18;
x23 = 2*x22;
x24 = cy1.*x16;
x25 = 2*x24;
x26 = -x23 - x25;
x27 = m1/2;
x28 = cy2.*x18;
x29 = cx2.*x6;
x30 = x16.*x29;
x31 = cz2.*x0;
x32 = x16.*x31;
x33 = -x28 + x30 + x32;
x34 = 2*x30;
x35 = 2*x32;
x36 = -2*x28 + x34 + x35;
x37 = cy2.*x16;
x38 = 2*x37;
x39 = x18.*x29;
x40 = 2*x39;
x41 = x18.*x31;
x42 = 2*x41;
x43 = -x

In [21]:
# --- VALIDAÇÃO FINAL ---
inputs_joints = ['Rz', 'Rz']
inputs_links = [[1, 0, 0], [1, 0, 0]]

# Instancia o Novo Motor Corrigido
bot = RobotMathEngine(inputs_joints, inputs_links)
bot.step_1_kinematics()
bot.step_2_jacobian()
bot.step_3_dynamics()

print("\n--- COMPARANDO COM NIKU (Frações Exatas) ---")
m1, m2, l1, l2 = sp.symbols('m1 m2 L1 L2')
C2 = sp.cos(bot.q[1])

# Matriz Alvo Niku (Exata)
M11_n = sp.Rational(1,3)*m1*l1**2 + m2*l1**2 + sp.Rational(1,3)*m2*l2**2 + m2*l1*l2*C2
M12_n = sp.Rational(1,3)*m2*l2**2 + sp.Rational(1,2)*m2*l1*l2*C2
M22_n = sp.Rational(1,3)*m2*l2**2

M_Niku = sp.Matrix([[M11_n, M12_n], [M12_n, M22_n]])

# Substituições
subs_map = {
    sp.symbols('L1'): l1, sp.symbols('m1'): m1,
    sp.symbols('L2'): l2, sp.symbols('m2'): m2,
    sp.symbols('cx1'): l1/2, sp.symbols('cx2'): l2/2, # CM no meio
    sp.symbols('cy1'): 0, sp.symbols('cz1'): 0,
    sp.symbols('cy2'): 0, sp.symbols('cz2'): 0,
    sp.symbols('Izz1'): sp.Rational(1,12)*m1*l1**2,
    sp.symbols('Izz2'): sp.Rational(1,12)*m2*l2**2,
    sp.symbols('Ixx1'): 0, sp.symbols('Iyy1'): 0,
    sp.symbols('Ixx2'): 0, sp.symbols('Iyy2'): 0
}

# Subtração
Diff = sp.simplify(sp.expand(bot.M.subs(subs_map) - M_Niku))

print("Resultado da Diferença:")
sp.pprint(Diff)

if Diff == sp.zeros(2,2):
    print("\n✅ SUCESSO! A validação bateu perfeitamente.")
else:
    print("\n❌ AINDA TEM ERRO.")

1. Calculando Cinemática (Correção Geométrica aplicada)...
2. Calculando Jacobiano...
3. Calculando Dinâmica...
   Separando termos...

--- COMPARANDO COM NIKU (Frações Exatas) ---
Resultado da Diferença:
⎡0  0⎤
⎢    ⎥
⎣0  0⎦

✅ SUCESSO! A validação bateu perfeitamente.
